# Bias detection and mitigation in AutoAI

This notebook contains the steps and code to demonstrate support of AutoAI experiments with bias detection/mitigation in watsonx.ai service. It introduces commands for data retrieval, training experiments, persisting pipelines, testing pipelines and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.12.


## Learning goals

The learning goals of this notebook are:

-  Work with watsonx.ai experiments to train AutoAI models.
-  Compare trained models quality and select the best one for further refinement.
-  Investigate models fairness.
-  Online deployment and score the trained model.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Optimizer definition](#definition)
3. [Experiment with bias detection run](#run)
3. [Experiment with bias mitigation run](#run_mit)
5. [Deploy and Score](#scoring)
6. [Clean up](#cleanup)
7. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pak for Data administrator and ask them for your account credentials

### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install -U wget | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U autoai-libs | tail -n 1
%pip install -U "lale[fairness]" | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [2]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

Enter your watsonx.ai password and hit enter:  ········


#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [5]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai, you need to set the **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="definition"></a>
## Optimizer definition

### Training data connection

This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv).

In [7]:
filename = "german_credit_data_biased_training.csv"

Download training data from git repository and create data assets.

In [8]:
import os
import wget

url = "https://github.com/IBM/watsonx-ai-samples/raw/master/cpd5.2/data/bias/german_credit_data_biased_training.csv"
if not os.path.isfile(filename):
    wget.download(url)

asset_details = client.data_assets.create(name=filename, file_path=filename)

Creating data asset...
SUCCESS


In [9]:
from ibm_watsonx_ai.helpers import DataConnection, AssetLocation

credit_risk_conn = DataConnection(
    location=AssetLocation(asset_id=client.data_assets.get_id(asset_details))
)

training_data_reference = [credit_risk_conn]

In [10]:
import pandas as pd

pd.read_csv(filename).head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


### Bias detection: optimizer configuration

Provide input information for AutoAI optimizer:
- `name` - experiment name
- `prediction_type` - type of the problem
- `prediction_column` - target column name
- `scoring` - optimization metric

Define `fairness_info`:

 - protected_attribute_names (array of items : string) – Subset of feature names for which fairness is desired.

 - favorable_labels (array of union) – Label values which are considered favorable (i.e. “positive”). Available types
        string
            Literal value
        number
            Numerical value
        array of number, >= 2 items, <= 2 items
            Numeric range [a,b] from a to b inclusive.

In [11]:
fairness_info = {
    "protected_attributes": [
        {"feature": "Sex", "reference_group": ["male"], "monitored_group": ["female"]},
        {
            "feature": "Age",
            "reference_group": [[20, 40], [60, 90]],
            "monitored_group": [[41, 59]],
        },
    ],
    "favorable_labels": ["No Risk"],
}

In [12]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, space_id=space_id)

pipeline_optimizer = experiment.optimizer(
    name="Credit Risk Prediction and bias detection - AutoAI",
    prediction_type=AutoAI.PredictionType.BINARY,
    prediction_column="Risk",
    scoring=AutoAI.Metrics.ACCURACY_SCORE,
    fairness_info=fairness_info,
    max_number_of_estimators=4,
)

Configuration parameters can be retrieved via `get_params()`.

In [13]:
pipeline_optimizer.get_params()

{'name': 'Credit Risk Prediction and bias detection - AutoAI',
 'desc': '',
 'prediction_type': 'binary',
 'prediction_column': 'Risk',
 'prediction_columns': None,
 'timestamp_column_name': None,
 'scoring': 'accuracy',
 'holdout_size': None,
 'max_num_daub_ensembles': 4,
 't_shirt_size': 'm',
 'train_sample_rows_test_size': None,
 'include_only_estimators': None,
 'include_batched_ensemble_estimators': None,
 'backtest_num': None,
 'lookback_window': None,
 'forecast_window': None,
 'backtest_gap_length': None,
 'cognito_transform_names': None,
 'csv_separator': ',',
 'excel_sheet': None,
 'encoding': 'utf-8',
 'positive_label': None,
 'drop_duplicates': True,
 'outliers_columns': None,
 'text_processing': None,
 'word2vec_feature_number': None,
 'daub_give_priority_to_runtime': None,
 'text_columns_names': None,
 'sampling_type': None,
 'sample_size_limit': None,
 'sample_rows_limit': None,
 'sample_percentage_limit': None,
 'number_of_batch_rows': None,
 'n_parallel_data_connection

<a id="run"></a>
## Experiment run

Call the `fit()` method to trigger the AutoAI experiment. You can either use interactive mode (synchronous job) or background mode (asychronous job) by specifying `background_model=True`.

In [14]:
run_details = pipeline_optimizer.fit(
    training_data_reference=training_data_reference, background_mode=False
)

Training job 5ab6871d-e40b-4204-aa64-81611f60f36d completed: 100%|████████| [04:14<00:00,  2.54s/it]


You can use the `get_run_status()` method to monitor AutoAI jobs in background mode.

In [15]:
pipeline_optimizer.get_run_status()

'completed'

Get experiment `training_id`.

In [16]:
training_id = run_details["metadata"]["id"]

### Get selected pipeline model

Download and reconstruct a scikit-learn pipeline model object from the
AutoAI training job.

In [17]:
pipeline_optimizer.summary()

,Enhancements,Estimator,training_accuracy_(optimized),training_disparate_impact_Sex,training_disparate_impact,training_roc_auc,holdout_disparate_impact_Sex,holdout_average_precision,holdout_log_loss,holdout_roc_auc,...,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1,holdout_disparate_impact,training_balanced_accuracy,holdout_disparate_impact_Age,training_f1,training_disparate_impact_Age
Pipeline Name,,,,,,,,,,,,,,,,,,,,,
Pipeline_11,,RandomForestClassifier,0.765781,1.105232,0.178571,0.819032,1.130991,0.453480,0.260547,0.825085,...,0.810360,0.923848,0.913011,0.845636,0.942943,0.481636,0.726539,0.514652,0.827574,0.216177
Pipeline_12,HPO,RandomForestClassifier,0.765781,1.105232,0.178571,0.819032,1.130991,0.453480,0.260547,0.825085,...,0.810360,0.923848,0.913011,0.845636,0.942943,0.481636,0.726539,0.514652,0.827574,0.216177
Pipeline_13,"HPO, FE",RandomForestClassifier,0.777827,1.104047,0.453209,0.829531,1.145059,0.454138,0.259776,0.799717,...,0.816122,0.915832,0.904011,0.859393,0.936937,0.506985,0.737750,0.516910,0.837181,0.265509
Pipeline_15,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(RandomForestClas...,0.777827,1.104047,0.453209,0.829531,1.145059,0.454138,0.259776,0.799717,...,0.816122,0.915832,0.904011,0.859393,0.936937,0.506985,0.737750,0.516910,0.837181,0.265509
Pipeline_14,"HPO, FE, HPO",RandomForestClassifier,0.777827,1.104047,0.453209,0.829531,1.145059,0.454138,0.259776,0.799717,...,0.816122,0.915832,0.904011,0.859393,0.936937,0.506985,0.737750,0.516910,0.837181,0.265509
Pipeline_6,,SnapRandomForestClassifier,0.783850,1.075318,0.000000,0.825015,0.999727,0.457421,0.308571,0.820630,...,0.803721,0.889780,0.845745,0.892953,0.921986,0.428484,0.730244,0.522214,0.845973,0.036683
Pipeline_1,,XGBClassifier,0.795228,1.022807,0.000000,0.842679,0.977185,0.465626,0.347874,0.841624,...,0.813766,0.833667,0.782745,0.897315,0.882270,0.359572,0.745069,0.495790,0.853488,0.037335
Pipeline_3,"HPO, FE",XGBClassifier,0.800581,1.010840,0.000000,0.847321,0.986677,0.478683,0.429919,0.838828,...,0.802877,0.807615,0.745311,0.927853,0.865922,0.407455,0.738052,0.500699,0.860838,0.105128
Pipeline_2,HPO,XGBClassifier,0.806381,0.999427,0.000000,0.849589,1.042510,0.480182,0.433762,0.837494,...,0.807340,0.807615,0.745311,0.930873,0.865922,0.450967,0.745216,0.483898,0.864715,0.071577


In [18]:
best_pipeline = pipeline_optimizer.get_pipeline()

### Visualize pipeline

In [19]:
best_pipeline.export_to_sklearn_pipeline()

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140045854714304',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'float_int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'c...
                                                                  OptStandardScaler(use_scaler_flag=False)),
                                                                 ('float32_transform',
                                                                  float32_transform())]))])),
                ('numpypermutearray',
                 NumpyPermuteArray(axis=0,
                                   permutation_indices=[0, 1, 2, 3, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19,
                                                        4])),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        max_features=None, n_estimators=10,
                                        n_jobs=1, random_state=33))])

Each node in the visualization is a machine-learning operator
(transformer or estimator). Each edge indicates data flow (transformed
output from one operator becomes input to the next).  The input to the
root nodes is the initial dataset and the output from the sink node
is the final prediction.  When you hover the mouse pointer over a
node, a tooltip shows you the configuration arguments of the
corresponding operator (tuned hyperparameters). When you click on the
hyperlink of a node, it brings you to a documentation page for the
operator.

### Reading training data

In [20]:
train_df = pipeline_optimizer.get_data_connections()[0].read()

train_X = train_df.drop(["Risk"], axis=1).values
train_y = train_df.Risk.values

### Test pipeline model locally

In [21]:
predicted_y = best_pipeline.predict(train_X)
predicted_y[:5]

array(['No Risk', 'No Risk', 'No Risk', 'No Risk', 'Risk'], dtype=object)

Define `fairness_info` for `disparate_impact` scorer.

In [22]:
fairness_info_lale = {
    "protected_attributes": [
        {"feature": train_df.columns.get_loc("Sex"), "reference_group": ["male"]},
        {
            "feature": train_df.columns.get_loc("Age"),
            "reference_group": [[20, 40], [60, 90]],
        },
    ],
    "favorable_labels": ["No Risk"],
}

In [23]:
from lale.lib.aif360 import disparate_impact
from sklearn.metrics import accuracy_score

disparate_impact_scorer = disparate_impact(**fairness_info_lale)
print("Accuracy: ", accuracy_score(y_true=train_y, y_pred=predicted_y))
print("Disparate impact: ", disparate_impact_scorer(best_pipeline, train_X, train_y))

Accuracy:  0.93014853472501
Disparate impact:  0.6918131000032457


## Bias mitigation: optimizer configuration

Provide input information for AutoAI optimizer:
- `name` - experiment name
- `prediction_type` - type of the problem
- `prediction_column` - target column name
- `scoring` - optimization metric

Define `fairness_info`:

 - protected_attribute_names (array of items : string) – Subset of feature names for which fairness is desired.

 - favorable_labels (array of union) – Label values which are considered favorable (i.e. “positive”). Available types
        string
            Literal value
        number
            Numerical value
        array of number, >= 2 items, <= 2 items
            Numeric range [a,b] from a to b inclusive.

In [24]:
fairness_info_mitigation = {
    "protected_attributes": [
        {"feature": "Sex", "reference_group": ["male"], "monitored_group": ["female"]}
    ],
    "favorable_labels": ["No Risk"],
}

In [25]:
experiment_mitigation = AutoAI(credentials, space_id=space_id)

pipeline_optimizer_mitigation = experiment_mitigation.optimizer(
    name="Credit Risk Prediction and bias mitigation - AutoAI",
    prediction_type=AutoAI.PredictionType.BINARY,
    prediction_column="Risk",
    scoring="accuracy_and_disparate_impact",
    fairness_info=fairness_info_mitigation,
    max_number_of_estimators=4,
)

Configuration parameters can be retrieved via `get_params()`.

In [26]:
pipeline_optimizer_mitigation.get_params()

{'name': 'Credit Risk Prediction and bias mitigation - AutoAI',
 'desc': '',
 'prediction_type': 'binary',
 'prediction_column': 'Risk',
 'prediction_columns': None,
 'timestamp_column_name': None,
 'scoring': 'accuracy_and_disparate_impact',
 'holdout_size': None,
 'max_num_daub_ensembles': 4,
 't_shirt_size': 'm',
 'train_sample_rows_test_size': None,
 'include_only_estimators': None,
 'include_batched_ensemble_estimators': None,
 'backtest_num': None,
 'lookback_window': None,
 'forecast_window': None,
 'backtest_gap_length': None,
 'cognito_transform_names': None,
 'csv_separator': ',',
 'excel_sheet': None,
 'encoding': 'utf-8',
 'positive_label': None,
 'drop_duplicates': True,
 'outliers_columns': None,
 'text_processing': None,
 'word2vec_feature_number': None,
 'daub_give_priority_to_runtime': None,
 'text_columns_names': None,
 'sampling_type': None,
 'sample_size_limit': None,
 'sample_rows_limit': None,
 'sample_percentage_limit': None,
 'number_of_batch_rows': None,
 'n_pa

<a id="run_mit"></a>
## Experiment run

Call the `fit()` method to trigger the AutoAI experiment. You can either use interactive mode (synchronous job) or background mode (asychronous job) by specifying `background_model=True`.

In [27]:
run_details_mitigation = pipeline_optimizer_mitigation.fit(
    training_data_reference=training_data_reference, background_mode=False
)

Training job 4b07ca7a-e84e-4560-bdea-791e547e1b0f completed: 100%|████████| [05:26<00:00,  3.26s/it]


You can use the `get_run_status()` method to monitor AutoAI jobs in background mode.

In [28]:
pipeline_optimizer_mitigation.get_run_status()

'completed'

### Get selected pipeline model

Download and reconstruct a scikit-learn pipeline model object from the
AutoAI training job.

In [29]:
pipeline_optimizer_mitigation.summary()

,Enhancements,Estimator,training_accuracy_and_disparate_impact_(optimized),training_disparate_impact_Sex,training_roc_auc,holdout_average_precision,holdout_log_loss,holdout_roc_auc,training_balanced_accuracy,training_f1,...,training_average_precision,training_log_loss,holdout_recall,training_precision,holdout_accuracy,holdout_balanced_accuracy,training_recall,holdout_f1,training_accuracy,holdout_disparate_impact
Pipeline Name,,,,,,,,,,,,,,,,,,,,,
Pipeline_12,"HPO, FE",RandomForestClassifier,0.849066,1.082927,0.823367,0.451629,0.241300,0.853050,0.736529,0.833775,...,0.893375,0.522056,0.951807,0.817159,0.935872,0.927999,0.851670,0.951807,0.774483,1.055582
Pipeline_14,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(RandomForestClas...,0.849066,1.082927,0.823367,0.451629,0.241300,0.853050,0.736529,0.833775,...,0.893375,0.522056,0.951807,0.817159,0.935872,0.927999,0.851670,0.951807,0.774483,1.055582
Pipeline_13,"HPO, FE, HPO",RandomForestClassifier,0.849066,1.082927,0.823367,0.451629,0.241300,0.853050,0.736529,0.833775,...,0.893375,0.522056,0.951807,0.817159,0.935872,0.927999,0.851670,0.951807,0.774483,1.055582
Pipeline_18,"HPO, FE, HPO",SnapRandomForestClassifier,0.879795,1.012826,0.811234,0.452716,0.258499,0.829549,0.711770,0.839352,...,0.888033,0.510205,0.987952,0.790114,0.923848,0.892179,0.895295,0.945245,0.772249,1.046470
Pipeline_19,"HPO, FE, HPO, Ensemble",BatchedTreeEnsembleClassifier(SnapRandomForest...,0.879795,1.012826,0.811234,0.452716,0.258499,0.829549,0.711770,0.839352,...,0.888033,0.510205,0.987952,0.790114,0.923848,0.892179,0.895295,0.945245,0.772249,1.046470
Pipeline_17,"HPO, FE",SnapRandomForestClassifier,0.879795,1.012826,0.811234,0.452716,0.258499,0.829549,0.711770,0.839352,...,0.888033,0.510205,0.987952,0.790114,0.923848,0.892179,0.895295,0.945245,0.772249,1.046470
Pipeline_16,HPO,SnapRandomForestClassifier,0.845783,1.097919,0.830050,0.453654,0.279788,0.853212,0.726712,0.842762,...,0.901254,0.484691,0.987952,0.802087,0.911824,0.874215,0.887913,0.937143,0.779834,0.939266
Pipeline_15,,SnapRandomForestClassifier,0.845783,1.097919,0.830050,0.453654,0.279788,0.853212,0.726712,0.842762,...,0.901254,0.484691,0.987952,0.802087,0.911824,0.874215,0.887913,0.937143,0.779834,0.939266
Pipeline_1,,XGBClassifier,0.889312,1.008463,0.843840,0.462943,0.330309,0.840235,0.740480,0.846126,...,0.911090,0.447442,0.948795,0.813743,0.861723,0.818709,0.881535,0.901288,0.786973,0.976996


In [30]:
best_pipeline_mitigation = pipeline_optimizer_mitigation.get_pipeline()

### Visualize pipeline

In [31]:
best_pipeline_mitigation.export_to_sklearn_pipeline()

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('float32_transform_140043319445360',
                                                 Pipeline(steps=[('numpycolumnselector',
                                                                  NumpyColumnSelector(columns=[0,
                                                                                               1,
                                                                                               2,
                                                                                               3,
                                                                                               5,
                                                                                               6,
                                                                                               7,
                                                                                               8,
                                                                                               9,
                                                                                               10,
                                                                                               11,
                                                                                               12,
                                                                                               13,
                                                                                               14,
                                                                                               15,
                                                                                               16,
                                                                                               17,
                                                                                               18,
                                                                                               19])),
                                                                 ('compressstrings',
                                                                  CompressStrings(compress_type='hash',
                                                                                  dtypes_list=['char_str',
                                                                                               'float_int_num',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'char_str',
                                                                                               'c...
                 autoai_libs.cognito.transforms.transform_utils.NSFA(significance = [0.0, np.float64(0.0454), 0.0, 0.0, 0.0, 0.0, np.float64(0.0469), 0.0, 0.0, np.float64(0.0379), np.float64(0.0845), np.float64(0.0313), np.float64(0.0732), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, np.float64(0.0415), 0.0, np.float64(0.0498), 0.0, 0.0, 0.0, 0.0, np.float64(0.0259), 0.0, np.float64(0.0422), np.float64(0.0271), 0.0, np.float64(0.0304), np.float64(0.0291), np.float64(0.0294), np.float64(0.0411), np.float64(0.0324), np.float64(0.0577), np.float64(0.0308), np.float64(0.0403)], protected_cols = [8], analyzer=sklearn.decomposition._pca.PCA(copy = True, iterated_power = 'auto', n_components = None, n_oversamples = 10, power_iteration_normalizer = 'auto', random_state = None, svd_solver = 'auto', tol = 0.0, whiten = False))),
                ('randomforestclassifier',
                 RandomForestClassifier(class_weight='balanced', max_depth=10,
                                        max_features=None, n_estimators=10,
                                        n_jobs=1, random_state=33))]

### Reading training data

In [32]:
train_df = pipeline_optimizer_mitigation.get_data_connections()[0].read()

train_X = train_df.drop(["Risk"], axis=1).values
train_y = train_df.Risk.values

### Test pipeline model locally

In [33]:
predicted_y = best_pipeline_mitigation.predict(train_X)
predicted_y[:5]

array(['No Risk', 'No Risk', 'No Risk', 'No Risk', 'Risk'], dtype=object)

Define `fairness_info` for `disparate_impact` and `accuracy_and_disparate_impact` scorers.

In [34]:
fairness_info_mitigation_lale = {
    "protected_attributes": [
        {"feature": train_df.columns.get_loc("Sex"), "reference_group": ["male"]}
    ],
    "favorable_labels": ["No Risk"],
}

In [35]:
from lale.lib.aif360 import disparate_impact, accuracy_and_disparate_impact
from sklearn.metrics import accuracy_score

disparate_impact_scorer = disparate_impact(**fairness_info_mitigation_lale)
accuracy_and_disparate_impact_scorer = accuracy_and_disparate_impact(
    **fairness_info_mitigation_lale
)
print("Accuracy: ", accuracy_score(y_true=train_y, y_pred=predicted_y))
print(
    "Accuracy and disparate impact: ",
    accuracy_and_disparate_impact_scorer(best_pipeline, train_X, train_y),
)
print("Disparate impact: ", disparate_impact_scorer(best_pipeline, train_X, train_y))

Accuracy:  0.9321557607386591
Accuracy and disparate impact:  0.9138764646804844
Disparate impact:  1.1140765419331191


<a id="scoring"></a>
## Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

In [36]:
pipeline_name = "Pipeline_1"

### Deployment creation

In [37]:
from ibm_watsonx_ai.deployment import WebService

service = WebService(credentials, source_space_id=space_id)

service.create(
    experiment_run_id=training_id,
    model=pipeline_name,
    deployment_name="Credit Risk Deployment AutoAI - bias detection",
)

Preparing an AutoAI Deployment...
Published model uid: 85b04bbc-496e-4eeb-aa28-1e4b4e6b8a6f
Deploying model 85b04bbc-496e-4eeb-aa28-1e4b4e6b8a6f using V4 client.


######################################################################################

Synchronous deployment creation for id: '85b04bbc-496e-4eeb-aa28-1e4b4e6b8a6f' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.......
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='56d14f2b-1d2f-4af2-ae42-6ee6aceed7a6'
-----------------------------------------------------------------------------------------------




Deployment object could be printed to show basic information:

In [ ]:
print(service)

To show all available information about deployment use `.get_params()` method:

In [ ]:
service.get_params()

### Scoring of webservice
You can make scoring request by calling `score()` on deployed pipeline.

In [40]:
predictions = service.score(payload=train_df.drop(["Risk"], axis=1).iloc[:10])
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.9059737920761108, 0.09402620792388916]],
    ['No Risk', [0.9039297699928284, 0.09607024490833282]],
    ['No Risk', [0.8551719188690186, 0.14482809603214264]],
    ['No Risk', [0.7936931848526001, 0.2063068449497223]],
    ['Risk', [0.11071383953094482, 0.8892861604690552]],
    ['Risk', [0.035136282444000244, 0.9648637175559998]],
    ['No Risk', [0.8070950508117676, 0.19290491938591003]],
    ['No Risk', [0.821358323097229, 0.17864170670509338]],
    ['No Risk', [0.9476691484451294, 0.05233084037899971]],
    ['Risk', [0.01895737648010254, 0.9810426235198975]]]}]}

If you want to work with the webservice in external Python application you can retrieve the service object by:
 - initialize service by `service = WebService(credentials)`
 - get deployment_id by `service.list()` method
 - get webservice object by `service.get('deployment_id')` method

After that you can call `service.score()` method.

### Deleting deployment
You can delete exeiting deployment by calling `service.delete()` command.
To list existing web services you can use `service.list()`.

<a id="cleanup"></a>
## Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!
 
You learned how to use `ibm-watsonx-ai` to run AutoAI experiments. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts.  

### Authors

**Dorota Lączak**, Software Engineer at watsonx.ai.

**Szymon Kucharczyk**, Software Engineer at watsonx.ai.

Copyright © 2023-2025 IBM. This notebook and its source code are released under the terms of the MIT License.